In [1]:
import argparse
import networkx as nx
import numpy as np
import scipy as sp
import tensorflow as tf
import tensorflow.contrib.eager as eager
import my
import objectives
import tf_gnn

/home/gaiyu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
tf.enable_eager_execution(config)

In [3]:
args = argparse.Namespace()
args.depth = 10
args.device = '/cpu:0'
# args.device = '/device:GPU:0'
# args.graph = 'soc-Epinions1'
# args.graph = 'soc-Slashdot0811'
# args.graph = 'soc-Slashdot0902'
args.graph = 'barabasi-albert-100-3'
args.lambda_node = 0.0
args.lambda_edge = 0.0
args.lambda_entropy = 0.0
args.n_features = 16
args.n_iterations = 500
args.n_machines = 10
args.radius = 3

# parser = argparse.ArgumentParser()
# parser.add_argument('--depth', type=int, default=10)
# parser.add_argument('--device', type=str, default='/cpu:0')
# parser.add_argument('--graph', type=str, default='soc-Epinions1')
# parser.add_argument('--lambda_node', type=float, default=1)
# parser.add_argument('--lambda_edge', type=float, default=1)
# parser.add_argument('--lambda_entropy', type=float, default=1)
# parser.add_argument('--n-features', type=int, default=16)
# parser.add_argument('--n-iterations', type=int, default=500)
# parser.add_argument('--n-machines', type=int, default=10)
# parser.add_argument('--radius', type=int, default=3)
# args = parser.parse_args()

keys = sorted(vars(args).keys())
run_id = 'ipynb-' + '-'.join('%s-%s' % (key, str(getattr(args, key))) for key in keys if key != 'device')
writer = tf.contrib.summary.create_file_writer('runs/' + run_id)
writer.set_as_default()

In [4]:
if args.graph.startswith('barabasi-albert'):
    n, m = tuple(map(int, args.graph.split('-')[-2:]))
    g = nx.barabasi_albert_graph(n, m)
else:
    g = my.read_edgelist('data/' + args.graph, fill=True)
adj = nx.adj_matrix(g)

In [5]:
with tf.device(args.device):
    objective = objectives.Objective(adj, args.n_machines, args.lambda_node, args.lambda_edge, args.lambda_entropy)
    gnn = tf_gnn.GNN((1,) + (args.n_features,) * args.depth, args.n_machines, adj, args.radius)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)

In [6]:
with tf.device(args.device):
    x = tf.constant(adj.sum(1), dtype=tf.float32)
    x -= tf.reduce_mean(x)
    x /= tf.sqrt(tf.reduce_mean(tf.square(x)))

In [7]:
global_step = tf.train.get_or_create_global_step()
with tf.device(args.device):
    for i in range(args.n_iterations):
        global_step.assign_add(1)
        with tf.contrib.summary.record_summaries_every_n_global_steps(1):
            with eager.GradientTape() as tape:
                z = gnn(x, training=True)
                rslt, r, b_node, b_edge, entropy = objective(z, training=True)
            gradients = tape.gradient(rslt, gnn.variables)
            optimizer.apply_gradients(zip(gradients, gnn.variables), global_step=tf.train.get_or_create_global_step())
            
            tf.contrib.summary.scalar('objective-log_p', rslt)
            r, b_node, b_edge, entropy = objective(z)
            rslt = r + args.lambda_node * b_node + args.lambda_edge * b_edge + args.lambda_entropy * entropy
            tf.contrib.summary.scalar('objective', rslt)
            tf.contrib.summary.scalar('replication-factor', r)
            tf.contrib.summary.scalar('node-balancedness', b_node)
            tf.contrib.summary.scalar('edge-balancedness', b_edge)
            tf.contrib.summary.scalar('entropy', entropy)
            
            if (i + 1) % 10 == 0:
                print('[iteration %d]%f %f %f %f %f' % (i + 1, rslt, r, b_node, b_edge, entropy))

[iteration 10]4.190932 4.190932 -2.286214 -2.223595 1.559521
[iteration 20]3.919498 3.919498 -2.290181 -2.140514 1.393119
[iteration 30]4.144146 4.144146 -2.291087 -2.216047 1.560906
[iteration 40]4.313107 4.313107 -2.293209 -2.218606 1.739156
[iteration 50]4.402010 4.402010 -2.301945 -2.243014 1.823789
[iteration 60]3.998550 3.998550 -2.289599 -2.146967 1.451083
[iteration 70]4.264457 4.264457 -2.298118 -2.211026 1.725836
[iteration 80]4.226362 4.226362 -2.297187 -2.196095 1.690964
[iteration 90]4.167989 4.167989 -2.296689 -2.180760 1.640067
[iteration 100]4.181338 4.181338 -2.293306 -2.186884 1.648436
[iteration 110]4.225783 4.225783 -2.292672 -2.197277 1.682503
[iteration 120]4.273950 4.273950 -2.290241 -2.205857 1.718163
[iteration 130]4.245172 4.245172 -2.286363 -2.195933 1.709426
[iteration 140]4.099951 4.099951 -2.285414 -2.156809 1.604972
[iteration 150]4.094145 4.094145 -2.283896 -2.154907 1.594812
[iteration 160]4.027215 4.027215 -2.279845 -2.129882 1.537380
[iteration 170]4.

In [ ]:
# ckpt = eager.Checkpoint(model=gnn)
# ckpt.save('./models/gnn')